# Introduction

This script is used to gather the necessary data.
Things to be fixed:
* Token still needs to be refreshed by hand
* Choose time range yourself by inserting the right parameters
* Choose things you want to investigate
* Choose metrics of interest

# Initialization

## Import libraries

In [4]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
from datetime import datetime
from datetime import timedelta  
import time
import sys
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None 

## Set up parameters

In [2]:
##### Choose scope #####
scopeID = 'cot.smart_lighting'

In [4]:
##### Insert token and generate URL #####
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiJmMjMzNzRlMy0zNjZjLTRjOGQtYWU5MS0xZTE0YjRjZDZlZGIiLCJleHAiOjE1NTQxMTI3MzYsIm5iZiI6MCwiaWF0IjoxNTU0MTEyMTM2LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJiOGVjOTBhZC04MTU5LTQxM2MtYjg5YS1lOTgyOGRlOTc3MzgiLCJhdXRoX3RpbWUiOjE1NTQxMTA3MTAsInNlc3Npb25fc3RhdGUiOiI3MmI5NjI3My1lZmZhLTRmMGEtYjM3Ni03ZmRlOTZjYjQ5MDkiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.kRWxpajPkxla1JUZHXRDdsHM2ODWBkbezpU80vFTu0JZvrPrXrMDCdUPCiNv2S8QdNCxNO9Hjfv_I0SsrYpAmOEVZetgvggi1O4HPjkGY7sCHJs7zKTwnflmfElpXW1XaCatO346oVDCncMgBtclvu2uBZ6kNjpamlJHTsu1knmqVjijYCHfIa8Cg_cBpfDtCpsVLumooKTxEar2aVRqFn793ZwYYJaMTHSKqoJfnitsbtjomraj3xxEHYXtSERM9FjAkTGmOZNLF6kI8AtqwLVfS8_l5TXFXJI2I0ELz5bYsKA924bFwbA5tVZsdy7_ZulDcapCg6AXq5ONBVwg4A'

In [4]:
##### Specify time range #####
##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
startDate = datetime(2019, 2, 1, 1, 0)
endDate = datetime(2019, 3, 1, 1, 0)
startTime = int(startDate.timestamp())*1000;  #print(startTime)
endTime = int(endDate.timestamp())*1000;  #print(endTime)

# Weather station

In [5]:
thingID = 'davis.davis.weather.1'

### Choose metrics

In [6]:
#metricsList = ['environment.light','environment.temperature','weather.uv','environment.relativehumidity']
metricsList = ['weather.pressure', 'weather.rainrate', 'weather.windspeed']

### Run query

In [24]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    #print(response.content)
    data = response.json() 
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    weather_data = df[['time','value']]
#     print(weather_data)
    path=r'C:\Users\JeffG\Desktop\Case 1 - data\weather_data'
    weather_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

weather.pressure_2019-02
weather.rainrate_2019-02
weather.windspeed_2019-02


# Battery pack

In [2]:
#thingID = 'munisense.msup1g30034' # west facing
#thingID = 'munisense.msup1i70124' # south facing
thingID = 'munisense.msup1h90115' # east facing

### Choose metrics

In [3]:
metricsList = ['Power.BatteryState']
# metricsList = ['Power.BatteryState','Power.BatteryVoltHR','Power.PercentageRemaining','Power.TimeRemaining']

### Run query

In [8]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_'+startDate.strftime('%Y-%m'))
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    response.content
    data = response.json(); #print(data)
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    battery_data = df[['time','value']]
    print(battery_data)
    #path=r'C:\Users\Jeff\Dropbox\ICT-Elektronica\Thesis\scripts\battery_data_3' #vaste pc
#     path=r'C:\Users\JeffG\Desktop\Case 1 - data\battery_data_2' #Laptop
#     battery_data.to_csv(path+'\\'+metricsList[i]+'_'+startDate.strftime('%Y-%m')+'.csv')

Power.BatteryState_2019-02
                time value
0      1549008830188     5
1      1549009010192     5
2      1549009190146     5
3      1549009370198     5
4      1549009550156     5
5      1549009730206     5
6      1549009910161     5
7      1549010090213     5
8      1549010270219     5
9      1549010450225     5
10     1549010630170     5
11     1549010810225     5
12     1549010990175     5
13     1549011170235     5
14     1549011350188     5
15     1549011530359     5
16     1549011710189     5
17     1549011890180     5
18     1549012070191     5
19     1549012250275     5
20     1549012430201     5
21     1549012610254     5
22     1549012790214     5
23     1549012970257     5
24     1549013150218     5
25     1549013330262     5
26     1549013510212     5
27     1549013690267     0
28     1549013870226     0
29     1549014050275     0
...              ...   ...
10990  1551392984835     0
10991  1551393164798     0
10992  1551393344841     0
10993  1551393524806     0
1

In [69]:
def load_voltage(startDate, endDate, thingID, myToken):
    ##### Specify time range #####
    ##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
    startTime = int(startDate.timestamp())*1000;  #print(startTime)
    endTime = int(endDate.timestamp())*1000;  #print(endTime)

    ##### Choose scope #####
    scopeID = 'cot.smart_lighting'

    ##### Specify query #####
    metricID = 'Power.BatteryVoltHR'
    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    ##### Execute query #####
    response = requests.get(myUrlQuery, headers=headers)

    try:
        data = response.json(); 
        column_names = data['columns']; 
        values = data['values']
        df = pd.DataFrame.from_dict(values)
        df.columns = column_names
        df["value"] = pd.to_numeric(df.value, errors='coerce')

        ##### Clean up dataFrame #####
        # Rename the column
        df.rename(columns={'value':'Power.BatteryVoltHR'}, inplace=True)
        # Set timestamp as index
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.time, unit='ms')))
        df.drop(['time'], axis=1,inplace=True)
        message = 'All good'

        return df, message

    except ValueError:
        df = None
        errorMessage = "Decoding JSON has failed. Probably the token has expired"
        return df, errorMessage
    

In [6]:
startDate = datetime(2019, 2, 1, 1, 0)
endDate = datetime(2019, 3, 1, 1, 0)

In [7]:
load_voltage(startDate, endDate, thingID, myToken)

,geohash,sourceId,Power.BatteryVoltHR
time,,,
2019-02-01 08:13:50.188,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:16:50.192,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:19:50.146,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:22:50.198,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:25:50.156,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:28:50.206,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:31:50.161,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:34:50.213,u155k52gzz91,munisense.msup1h90115,13.66
2019-02-01 08:37:50.219,u155k52gzz91,munisense.msup1h90115,13.66


In [11]:
endDate = datetime.now()
endDate = endDate + timedelta(hours=1)
endDate

datetime.datetime(2019, 4, 1, 12, 49, 41, 326856)

In [12]:
startDate = endDate - timedelta(6*365/12)
window_size = 256
sigma = 3
fixed_std = 0.3

In [70]:
endDate = datetime.now()
startDate = endDate - timedelta(3/12*365)
thingID = 'munisense.msup1h90115' # east facing
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI1Y2Y3NjAyNi1mYTJhLTRmZTYtYWU5ZC02NDU1MjI3ZWVhZDciLCJleHAiOjE1NTQxNDc0MDQsIm5iZiI6MCwiaWF0IjoxNTU0MTQ2ODA0LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJhMzRmMmE1Yy02NzgzLTRmYmMtOTNkMy1kOWIyNDhhNTFmZmEiLCJhdXRoX3RpbWUiOjE1NTQxNDY4MDMsInNlc3Npb25fc3RhdGUiOiIwOTQyN2VkMC02YTg0LTRlZDktOTYyOC04NzkyMjAzZTM2ODkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.hz9g4pUg-_xFRWum7BR4hhjCW3mQEaBgQ3VfJe-IRoDoIa6FmDQ7J-AQ7W4PbkSr1cXw4sy94B4fL_6GYoV2QwdqJNEqipCV0YVvfNt7UmLeyLMLDPQbZIrRMu-O1k-JQ67nroFlPCQKrIdB5pvfqYHQzpEkp6niAQu_Mypm2Hgz7ulSC0xlgY3rqmkIo1VhpRNULEmS1B19_91DdFgmADX6843HtOlJCll4xJ-hLfpnFBNejczNX8gQ9ImfRzJxSYEAAwETQLVgDXu_gQyeo7z4P0OuWx51SDAhhHFp9f5tfjLDvb86Kk2sZxSbdsH9sMANAfzduMGjFVfZIzxOFA'

df = load_voltage(startDate, endDate, thingID, myToken)

In [30]:
df

'Deconding JSON has failed. Probably the token has expired'

In [14]:
df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()
df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']

In [15]:
def label_anomaly_moving_std (row):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * row['moving_std']))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * row['moving_std']))):
        return 1
    else:
        return 0

def label_anomaly_fixed_std (row, fixed_std):
    if (row['Power.BatteryVoltHR'] > (row['moving_average'] + (sigma * fixed_std))) | (row['Power.BatteryVoltHR'] < (row['moving_average'] - (sigma * fixed_std))):
        return 1
    else:
        return 0

In [16]:
def low_pass_filtering(df, window_size, sigma, fixed_std):
    # Calculate moving average
    df['moving_average'] = df['Power.BatteryVoltHR'].rolling(window_size).mean()

    # Calculate anomalies
    df['residual'] = df['Power.BatteryVoltHR'] - df['moving_average']
    df['moving_std'] = df['Power.BatteryVoltHR'].rolling(window_size).std()
    df.dropna(inplace=True)
    df['anomaly_flag_moving_std']=df.apply (lambda row: label_anomaly_moving_std(row), axis=1)
    df['anomaly_flag_fixed_std']=df.apply (lambda row: label_anomaly_fixed_std(row, fixed_std), axis=1)

    moving_anomalies = df[df['anomaly_flag_moving_std']==1]
    fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]
    
    return df

In [17]:
df = low_pass_filtering(df, 256, 3, fixed_std)
moving_anomalies = df[df['anomaly_flag_moving_std']==1]
fixed_anomalies = df[df['anomaly_flag_fixed_std']==1]

In [18]:
df.tail()

,geohash,sourceId,Power.BatteryVoltHR,moving_average,residual,moving_std,anomaly_flag_moving_std,anomaly_flag_fixed_std
time,,,,,,,,
2019-04-01 09:36:54.181,u155k52gzz91,munisense.msup1h90115,16.29,16.294570,-0.004570,0.014918,0,0
2019-04-01 09:39:54.176,u155k52gzz91,munisense.msup1h90115,16.28,16.294375,-0.014375,0.014779,0,0
2019-04-01 09:42:54.178,u155k52gzz91,munisense.msup1h90115,16.28,16.294219,-0.014219,0.014719,0,0
2019-04-01 09:45:54.175,u155k52gzz91,munisense.msup1h90115,16.28,16.294062,-0.014062,0.014656,0,0
2019-04-01 09:48:54.176,u155k52gzz91,munisense.msup1h90115,16.29,16.293945,-0.003945,0.014568,0,0


In [21]:
df.last_valid_index()

Timestamp('2019-04-01 09:48:54.176000')

In [49]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

In [52]:
trace1 = go.Scatter(
        x=df.index,
        y=df['Power.BatteryVoltHR'],
        mode='markers',
        opacity=0.7,
        marker={
        'size': 6
        },
        name='Raw data'
        )
trace2 = go.Scatter(
        x=df.index,
        y=df['moving_average'],
        mode='lines',
        name='Moving average'
        )
trace3= go.Scatter(
        x=fixed_anomalies.index,
        y=fixed_anomalies['Power.BatteryVoltHR'],
        mode='markers',
        marker={
        'size': 10,
        'symbol': 'x',
        'color': 'red'
        },
        name='anomalies'   
        )

In [56]:
fig = tools.make_subplots(rows=2, cols=1)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace3, 1, 1)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(height=600, width=800, title='i <3 annotations and subplots')
py.iplot(fig, filename='simple-subplot-with-annotations')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

The draw time for this plot will be slow for clients without much RAM.


C:\Users\JeffG\Anaconda3\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



# Saving data to SQL database

In [8]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [9]:
user = 'root'
passw = 'root'
host =  'localhost'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
port = 3306 
database = 'smart_lighting'

In [10]:
engine = create_engine('mysql+mysqlconnector://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

In [17]:
endDate = datetime.now()
startDate = endDate - timedelta(9/12*365)
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiIzOGZkM2UwMi1mOTE4LTQzNTMtYmQwZi1hNTE0MWUzZTc5ZjUiLCJleHAiOjE1NTQxMzIwMjEsIm5iZiI6MCwiaWF0IjoxNTU0MTMxNDIxLCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiIyNTUwNDA4YS03NWRjLTRiM2EtYjM2Yi1iYzQ5ZjU3MzhlMWYiLCJhdXRoX3RpbWUiOjE1NTQxMzA5MDAsInNlc3Npb25fc3RhdGUiOiIzNjFjMmUzYi1kMTRlLTRhZTMtODU1Ni1kOTA4OTFkMGM4OWUiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.d-QVLWkYSYwbBBsFaub3onbnrF5ikyrVhu6z4jCIXQm7g1-Khc-SFWcD802IM0QppseSQziFD0YnhrVTD4SipUncTiJBjT2db235pKk8715tLH1PvijCDnPiTESs8IB6EZWEdBuiInEbgGN-3shhixfWySTA6L1W0XpkHvGpYdbDgIaTSz_uHn6Ebe2DpLm6Nu5MR1teR1ojuz_jLZVC0KNr75iUSfINCecOo2blbzQqVP_E1B9vk2b9NBiAS5vwX_j3r8kw-NX-xbK9qVRMQl1ImC38QBEveqNDE2qIEK3kCP7VKBsZX2t1M5KvpZ3q7f6oWRm9lQc5QMoAh5ThGw'
things = ['munisense.msup1g30034', 'munisense.msup1i70124', 'munisense.msup1h90115', 'munisense.msup1h90103']
for thingID in things:
    df = load_voltage(startDate, endDate, thingID, myToken)
    df.to_sql(name='battery_voltage', con=engine, if_exists = 'append', index=True)

In [33]:
table_name = 'battery_voltage'
endDate = datetime.now()
startDate = endDate - timedelta(9/12*365)
end_date = endDate.strftime("%Y-%m-%d %H:%M:%S")
start_date = startDate.strftime("%Y-%m-%d %H:%M:%S")
query = ''' SELECT * 
                FROM {} 
                WHERE (sourceId = '{}') AND (time between '{}' and '{}')'''.format(table_name, thingID, startDate, endDate)
test = pd.read_sql(query, engine, index_col='time')

In [34]:
test

,geohash,sourceId,Power.BatteryVoltHR
time,,,
2018-08-31 09:50:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 09:53:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 09:56:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 09:59:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:02:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 10:05:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:08:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:11:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:14:16,u155k52gzz91,munisense.msup1g30034,16.00


In [ ]:
table_name = 'battery_voltage'
thingID = 'munisense.msup1g30034'
query = "SELECT * FROM {} WHERE sourceId = '{}'".format(table_name, thingID)
test = pd.read_sql(query, engine, index_col='time')

In [6]:
def initialize_database():
    user = 'root'
    passw = 'root'
    host =  'localhost'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
    port = 3306 
    database = 'smart_lighting'
    engine = create_engine('mysql+mysqlconnector://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)
    return engine

def load_voltage_from_database(thingID, startDate, endDate, engine):
    table_name = 'battery_voltage'
    start_date = startDate.strftime("%Y-%m-%d %H:%M:%S")
    end_date = endDate.strftime("%Y-%m-%d %H:%M:%S")
    query = ''' SELECT * 
                FROM {} 
                WHERE (sourceId = '{}') AND (time between '{}' and '{}')'''.format(table_name, thingID, start_date, end_date)
    df = pd.read_sql(query, engine, index_col='time')
    return df

In [7]:
engine = initialize_database()

NameError: name 'create_engine' is not defined

In [37]:
df = load_voltage_from_database(thingID, startDate, endDate, engine)

In [38]:
df

,geohash,sourceId,Power.BatteryVoltHR
time,,,
2018-08-31 09:50:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 09:53:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 09:56:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 09:59:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:02:16,u155k52gzz91,munisense.msup1g30034,15.99
2018-08-31 10:05:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:08:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:11:16,u155k52gzz91,munisense.msup1g30034,16.00
2018-08-31 10:14:16,u155k52gzz91,munisense.msup1g30034,16.00


In [49]:
table_name = 'battery_voltage'
thingID = 'munisense.msup1g30034'
query = ''' SELECT
              sourceId, MAX('time')
            FROM
              {}
            GROUP BY
              sourceId'''.format(table_name, thingID)
test = pd.read_sql(query, engine)

In [50]:
test

,sourceId,MAX('time')
0,munisense.msup1g30034,time
1,munisense.msup1h90103,time
2,munisense.msup1h90115,time
3,munisense.msup1i70124,time


In [2]:
def find_last_datapoint(thingID):
    table_name = 'battery_voltage'
    query = ''' SELECT 
                    time
                FROM 
                    {}
                WHERE 
                    sourceId = '{}'
                ORDER BY time DESC
                LIMIT 1'''.format(table_name, thingID)
    result = pd.read_sql(query, engine)
    return result['time'].iloc[0]

In [55]:
test['time']

0   2019-01-28 15:57:37
Name: time, dtype: datetime64[ns]

In [75]:
for thingID in things:
        startDate = find_last_datapoint(thingID)
        endDate = datetime.now()
        df, message = load_voltage(startDate, endDate, thingID, myToken)
        print(df)
#         df.to_sql(name='battery_voltage', con=engine, if_exists = 'append', index=True)

KeyError: 'columns'

In [73]:
startDate = find_last_datapoint(thingID)

In [74]:
startDate

Timestamp('2019-01-28 15:57:37')

In [ ]:
def load_voltage(startDate, endDate, thingID, myToken):
    ##### Specify time range #####
    ##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
    startTime = int(startDate.timestamp())*1000;  #print(startTime)
    endTime = int(endDate.timestamp())*1000;  #print(endTime)

    ##### Choose scope #####
    scopeID = 'cot.smart_lighting'

    ##### Specify query #####
    metricID = 'Power.BatteryVoltHR'
    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    ##### Execute query #####
    response = requests.get(myUrlQuery, headers=headers)

    try:
        data = response.json(); 
        column_names = data['columns']; 
        values = data['values']
        df = pd.DataFrame.from_dict(values)
        df.columns = column_names
        df["value"] = pd.to_numeric(df.value, errors='coerce')

        ##### Clean up dataFrame #####
        # Rename the column
        df.rename(columns={'value':'Power.BatteryVoltHR'}, inplace=True)
        # Set timestamp as index
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.time, unit='ms')))
        df.drop(['time'], axis=1,inplace=True)
        message = 'All good'

        return df, message

    except ValueError:
        df = None
        errorMessage = "Decoding JSON has failed. Probably the token has expired"
        return df, errorMessage

In [12]:
thingID = 'munisense.msup1i70124'
startDate = find_last_datapoint(thingID); print(startDate)
endDate = datetime.now(); print(endDate)
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiI1Y2U2ZTYxMS1kOWI2LTQ0ZjAtYjAzNi0wZDA0NjVlOWM3MGIiLCJleHAiOjE1NTQyMDk4NTAsIm5iZiI6MCwiaWF0IjoxNTU0MjA5MjUwLCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiI4NjQxNzVjNy1hZmEzLTQwNjUtODM2YS0xYTE4Y2E3N2E3MzQiLCJhdXRoX3RpbWUiOjE1NTQyMDkyNDksInNlc3Npb25fc3RhdGUiOiJkNjkzZGM3NS1hMzRkLTQyOGQtYjlhOS1jMzAwOGFkYTUzNTUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.HtvRSGgP-Sbl556Qp7gU4QdrIFEKIUaluFLJti0gZD74-2lGsqKoeiQZtNlNNYkpjmIV1BgtjqWmIvI2fBFqUHdufTOhxjbmFZQ05-iMnjm54eDzIzKJEwDwKvUjngn8DTuwlh4vMAyaiJ58Brh47Xk9qT-vUXALUajHpcpvKEsk3GACB6QWmCNw16bOGk6W3dI4Zs54TxYsWKvE2eaqBzy1yWsAolSmEt-Z_h4VBxW4SecQBJT0m1L-fvt1jKL2Adb9P_o0y2CeMFQ82zKCDcPpxLu7Bjkj4JuVJvR_5SNXAJQt4gY_YjHQTdQ0E6DTlhhMnHSL_CoobF-gVvoFmA'

2019-04-02 12:43:01
2019-04-02 14:47:36.189621


In [13]:
##### Specify time range #####
##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
startTime = int(startDate.timestamp())*1000;  print(startTime)
endTime = int(endDate.timestamp())*1000;  print(endTime)

    ##### Choose scope #####
scopeID = 'cot.smart_lighting'

    ##### Specify query #####
metricID = 'Power.BatteryVoltHR'
myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    ##### Execute query #####
response = requests.get(myUrlQuery, headers=headers)

1554208981000
1554209256000


In [14]:
data = response.json(); 

In [15]:
data

{'columns': ['time', 'geohash', 'sourceId', 'value'],
 'values': [[1554208981467, 'u155k52gzz91', 'munisense.msup1i70124', '16.24']]}